# For historical tornadoes, find closest NEXRAD radar site 

In [45]:
import numpy as np
import pandas as pd

## Read all the data
#### tornado file trimmed down from the 1950-2019 file at https://www.spc.noaa.gov/wcm/#data

In [46]:
tornadoes = pd.read_csv('~/Box/MPO624_data/1991-2019_all_tornadoes.csv')

In [47]:
tornadoes

,om,yr,mo,dy,date,time,tz,st,stf,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
0,1,1991,1,14,1/14/91,17:00:00,3,TX,48,1,...,1.00,100,1,1,1,287,0,0,0,0
1,2,1991,1,14,1/14/91,17:50:00,3,TX,48,2,...,0.20,20,1,1,1,21,0,0,0,0
2,3,1991,1,14,1/14/91,18:14:00,3,TX,48,3,...,5.00,60,1,1,1,21,0,0,0,0
3,4,1991,1,14,1/14/91,18:18:00,3,TX,48,4,...,1.50,250,1,1,1,89,0,0,0,0
4,5,1991,1,14,1/14/91,18:18:00,3,TX,48,5,...,0.50,50,1,1,1,481,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36366,618537,2019,12,29,12/29/19,16:03:00,3,MS,28,0,...,7.70,900,1,1,1,7,0,0,0,0
36367,618538,2019,12,29,12/29/19,16:13:00,3,MS,28,0,...,3.82,200,1,1,1,19,0,0,0,0
36368,618539,2019,12,29,12/29/19,16:32:00,3,MS,28,0,...,2.61,200,1,1,1,105,0,0,0,0
36369,618540,2019,12,29,12/29/19,17:13:00,3,MS,28,0,...,3.23,125,1,1,1,101,0,0,0,0


In [48]:
# RADAR site locations from https://en.wikipedia.org/wiki/NEXRAD#cite_note-54

sites = pd.read_table('~/Box/MPO624_data/WSR88d_locations.txt')

In [49]:
sites[0:5]

,State,Name,Code,Lat,WestLon
0,PR,San Juan,TJUA,18.115600,66.078064
1,ME,Houlton,KCBW,46.039194,67.806603
2,ME,Gray/Portland,KGYX,43.891356,70.256554
3,VT,Burlington,KCXX,44.510994,73.166424
4,MA,Boston\t,KBOX,41.955892,71.136968


## Loop over tornadoes, calculate all site distances

In [50]:
# Starting longitude: how to get the value as a number, from Pandas

tornadoes[23:24]['slon'].values[0]

-82.47

In [51]:
# Site longitude, how to get the value as a number, from Pandas

sites[11:12]['WestLon'].values[0]

74.4108027

# OK, ready for the magic loop
## For each tornado, calculate distance of START POINT to nearest radar 

Add two columns to tornados DataFrame
1. that distance (so we can sort for the smallest value)
2. radar ID code

In [40]:
# For speed of testing, create a subset 

subset = tornadoes[0:10].copy()
subset

,om,yr,mo,dy,date,time,tz,st,stf,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
0,1,1991,1,14,1/14/91,17:00:00,3,TX,48,1,...,1.0,100,1,1,1,287,0,0,0,0
1,2,1991,1,14,1/14/91,17:50:00,3,TX,48,2,...,0.2,20,1,1,1,21,0,0,0,0
2,3,1991,1,14,1/14/91,18:14:00,3,TX,48,3,...,5.0,60,1,1,1,21,0,0,0,0
3,4,1991,1,14,1/14/91,18:18:00,3,TX,48,4,...,1.5,250,1,1,1,89,0,0,0,0
4,5,1991,1,14,1/14/91,18:18:00,3,TX,48,5,...,0.5,50,1,1,1,481,0,0,0,0
5,6,1991,1,14,1/14/91,18:20:00,3,TX,48,6,...,0.2,20,1,1,1,481,0,0,0,0
6,7,1991,1,14,1/14/91,18:25:00,3,TX,48,7,...,0.2,20,1,1,1,481,0,0,0,0
7,8,1991,1,14,1/14/91,18:30:00,3,TX,48,8,...,0.2,20,1,1,1,481,0,0,0,0
8,9,1991,1,14,1/14/91,19:55:00,3,TX,48,9,...,0.6,13,1,1,1,321,0,0,0,0
9,10,1991,1,14,1/14/91,21:40:00,3,TX,48,10,...,0.2,10,1,1,1,167,0,0,0,0


In [41]:
# Loop and collect Code and Distance lists
NearestCode = []
NearestDist = []

for i in range(len(subset)): 
    # print(tornadoes[i:i+1]['st'].values[0]) # debugging/testing: print the state
    x = tornadoes[i:i+1]['slon'].values[0]
    y = tornadoes[i:i+1]['slat'].values[0]
    min_dist = 1e9 # start with a large value, and find the minimuum
    
    for j in range(len(sites)):
        # print(sites[j:j+1])
        # dx is distance eastward (using cos of latitude correction on sphere)
        dx = (x + sites[j:j+1]['WestLon'].values[0]) * np.cos(y*3.14/180.)
        dy = y - sites[j:j+1]['Lat'].values[0]
        dist2 = dx*dx + dy*dy
        
        if (dist2 < min_dist): 
            # print(sites[j:j+1], '   DISTANCE ', np.sqrt(dist2))
            min_dist = np.sqrt(dist2)
            closest = j 
    #print('') # debugging/testing
    #print('closest station, distance: ')
    #print(sites[closest:closest+1]['Code'].values[0])
    #print('DISTANCE (km): ', min_dist*111.111)
    #print('')
    NearestDist.append(min_dist*111.111) # km 
    NearestCode.append(sites[closest:closest+1]['Code'].values[0])

# Append columns onto the tornadoes DataFrame

In [42]:
subset['NearestRadar'] = NearestCode
subset['NearestDist'] = NearestDist

In [43]:
# build the URL

URLs = 'https://s3.amazonaws.com/noaa-nexrad-level2/index.html#' + tornadoes['date'] + '/' + subset['NearestRadar']
URLs[0]

'https://s3.amazonaws.com/noaa-nexrad-level2/index.html#1/14/91/KGRK'

#### Oh too bad, the date will have to be chopped up and reordered
###### needs to be in this format: https://s3.amazonaws.com/noaa-nexrad-level2/index.html#1997/06/06/KAMA/

In [44]:
subset.sort_values(by='NearestDist')[0:100]

,om,yr,mo,dy,date,time,tz,st,stf,stn,...,ns,sn,sg,f1,f2,f3,f4,fc,NearestRadar,NearestDist
9,10,1991,1,14,1/14/91,21:40:00,3,TX,48,10,...,1,1,1,167,0,0,0,0,KHGX,3.126000
0,1,1991,1,14,1/14/91,17:00:00,3,TX,48,1,...,1,1,1,287,0,0,0,0,KGRK,74.276494
2,3,1991,1,14,1/14/91,18:14:00,3,TX,48,3,...,1,1,1,21,0,0,0,0,KGRK,84.278825
1,2,1991,1,14,1/14/91,17:50:00,3,TX,48,2,...,1,1,1,21,0,0,0,0,KGRK,88.335741
7,8,1991,1,14,1/14/91,18:30:00,3,TX,48,8,...,1,1,1,481,0,0,0,0,KHGX,92.737076
6,7,1991,1,14,1/14/91,18:25:00,3,TX,48,7,...,1,1,1,481,0,0,0,0,KHGX,96.399705
5,6,1991,1,14,1/14/91,18:20:00,3,TX,48,6,...,1,1,1,481,0,0,0,0,KHGX,97.254585
4,5,1991,1,14,1/14/91,18:18:00,3,TX,48,5,...,1,1,1,481,0,0,0,0,KHGX,100.368893
8,9,1991,1,14,1/14/91,19:55:00,3,TX,48,9,...,1,1,1,321,0,0,0,0,KHGX,100.790364
3,4,1991,1,14,1/14/91,18:18:00,3,TX,48,4,...,1,1,1,89,0,0,0,0,KEWX,142.701384
